In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns 
from sklearn.metrics import mean_squared_error, r2_score

Make benchmark performance model

In [2]:
df = pd.read_csv('data/data_arima.csv', index_col=1)[['mood', 'benchmark']] ## todo change: do data that RF uses
df = df[~df.mood.isna() & ~df.benchmark.isna()] ## todo: remove after using data that RF uses
benchmark = df.groupby('id').apply(lambda x: mean_squared_error(x.mood, x.benchmark)).to_frame('mse')
benchmark['r2'] = df.groupby('id').apply(lambda x: r2_score(x.mood, x.benchmark))
benchmark['corr'] = df.apply(lambda x: abs(x.mood - x.benchmark) < 0.5, axis=1).groupby('id').apply(lambda x: x.sum() / x.count())
benchmark
benchmark.to_csv('results/benchmark.csv')